## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-03-10-44-00 +0000,bbc,Was it a coup or was it a 'sham'? Behind Guine...,https://www.bbc.com/news/articles/c1m8nm22785o...
1,2025-12-03-10-42-54 +0000,nyt,"Fabergé Winter Egg Sells for $30 Million, an A...",https://www.nytimes.com/2025/12/03/world/europ...
2,2025-12-03-10-42-16 +0000,bbc,Airbus to inspect some planes over 'quality is...,https://www.bbc.com/news/articles/c3v19q3r31ro...
3,2025-12-03-10-39-33 +0000,bbc,India scraps order to pre-install state-run cy...,https://www.bbc.com/news/articles/clydg2re4d1o...
4,2025-12-03-10-36-00 +0000,wsj,Shares in Zara Owner Inditex Jump on Strong Sa...,https://www.wsj.com/business/retail/zara-paren...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2431/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
38,trump,69
360,up,17
39,he,15
751,new,13
783,drug,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
169,2025-12-02-23-36-00 +0000,wsj,The Trump administration canceled interviews w...,https://www.wsj.com/economy/central-banking/tr...,215
120,2025-12-03-02-00-00 +0000,wsj,"Pete Hegseth, who won over Trump by defending ...",https://www.wsj.com/politics/national-security...,165
130,2025-12-03-01-41-00 +0000,wsj,President Trump on Tuesday lashed out against ...,https://www.wsj.com/politics/policy/ice-plans-...,132
334,2025-12-02-14-41-48 +0000,nypost,Former Honduran President Juan Orlando Hernand...,https://nypost.com/2025/12/02/world-news/forme...,124
285,2025-12-02-18-23-52 +0000,cbc,"Ex-Honduran president, convicted of traffickin...",https://www.cbc.ca/news/world/us-honduras-hern...,123


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
169,215,2025-12-02-23-36-00 +0000,wsj,The Trump administration canceled interviews w...,https://www.wsj.com/economy/central-banking/tr...
120,96,2025-12-03-02-00-00 +0000,wsj,"Pete Hegseth, who won over Trump by defending ...",https://www.wsj.com/politics/national-security...
68,67,2025-12-03-06-20-43 +0000,nypost,California girl was 2 weeks away from her 9th ...,https://nypost.com/2025/12/03/us-news/californ...
285,54,2025-12-02-18-23-52 +0000,cbc,"Ex-Honduran president, convicted of traffickin...",https://www.cbc.ca/news/world/us-honduras-hern...
167,54,2025-12-02-23-45-31 +0000,latimes,"Baby, baby, baby, baby, oh, baby: New gorilla ...",https://www.latimes.com/california/story/2025-...
274,52,2025-12-02-19-15-09 +0000,nypost,"Kraft, Coca-Cola among companies sued by San F...",https://nypost.com/2025/12/02/business/kraft-c...
102,51,2025-12-03-03-14-50 +0000,nypost,Marco Rubio claims ‘progress’ towards Ukraine-...,https://nypost.com/2025/12/02/us-news/marco-ru...
244,42,2025-12-02-20-29-40 +0000,latimes,Saudi Arabia's public investment fund to own a...,https://www.latimes.com/entertainment-arts/bus...
89,42,2025-12-03-04-10-00 +0000,wsj,Republican Matt Van Epps won the special elect...,https://www.wsj.com/politics/elections/republi...
103,38,2025-12-03-03-10-40 +0000,bbc,YouTube says it will be less safe for kids und...,https://www.bbc.com/news/articles/cp341gz24p5o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
